In [67]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

from nba_api.stats.endpoints import leagueseasonmatchups
from thefuzz import fuzz, process
player_dict = get_players_pbp()
pID_dict = get_pID_pbp()

def get_missing_pId(player,player_dict):
    pId = process.extract(player,player_dict,limit=1, scorer=fuzz.partial_ratio)[0][2]
    return pId

img_DIR_P = "../data/images/players/"
fig_DIR = "../figs/players/"

In [80]:
stats = leagueseasonmatchups.LeagueSeasonMatchups(
    league_id="00",
    per_mode_simple="Totals",
    season=get_ss(2023),
    season_type_playoffs="Playoffs"
)
df2 = stats.get_data_frames()[0]

In [81]:
url = "https://www.basketball-reference.com/playoffs/NBA_2024_advanced.html#advanced_stats::bpm"
url = "https://www.basketball-reference.com/leagues/NBA_2024_advanced.html"
df1 = pd.read_html(url)
df1 = df1[0]
df1["playerID"] = df1["Player"].map(pID_dict)
df1.loc[df1["playerID"].isna(),"playerID"] = df1.loc[df1["playerID"].isna(),"Player"].apply(lambda x: get_missing_pId(x,player_dict))
df1["playerID"] = df1["playerID"].astype(int)
df3 = pd.merge(df2,df1[["playerID","BPM","OBPM","DBPM"]],left_on="OFF_PLAYER_ID",right_on="playerID")
df3["BPM"] = df3["BPM"].astype(float)
df3["OBPM"] = df3["OBPM"].astype(float)
df3["DBPM"] = df3["DBPM"].astype(float)
df3["pBPM"] = df3["PARTIAL_POSS"]*df3["BPM"]
df3["pOBPM"] = df3["PARTIAL_POSS"]*df3["OBPM"]
df3["pDBPM"] = df3["PARTIAL_POSS"]*df3["DBPM"]

In [82]:
df4 = df3.groupby(["DEF_PLAYER_ID","DEF_PLAYER_NAME"])[["PARTIAL_POSS","pBPM","pOBPM","pDBPM"]].sum().reset_index()
df4["diff"] = round(df4["pBPM"]/df4["PARTIAL_POSS"],2)
df4["Odiff"] = round(df4["pOBPM"]/df4["PARTIAL_POSS"],2)
df4["Ddiff"] = round(df4["pDBPM"]/df4["PARTIAL_POSS"],2)

In [83]:
df4.columns

Index(['DEF_PLAYER_ID', 'DEF_PLAYER_NAME', 'PARTIAL_POSS', 'pBPM', 'pOBPM',
       'pDBPM', 'diff', 'Odiff', 'Ddiff'],
      dtype='object')

In [84]:
df5 = df4.query("PARTIAL_POSS > 50").sort_values("Odiff",ascending=False).reset_index(drop=True).reset_index()
df5["index"] +=1
df5 = df5[['index','DEF_PLAYER_ID', 'DEF_PLAYER_NAME', 'PARTIAL_POSS','Odiff']].head(10)
df5["DEF_PLAYER_ID"] = df5["DEF_PLAYER_ID"].astype(str)
df5["PARTIAL_POSS"] = df5["PARTIAL_POSS"].round(1)

In [85]:
t = (
    GT(df5)
    .tab_header(
        title=md("Highest Matchup Difficulty: Weighted by Regular Season OBPM"),
        subtitle="Min 50 Partial Possessions Defended"
    )
    .tab_source_note(source_note="@SravanNBA | source: nba.com/stats via nba_api")
    .cols_label(
        index = "#",
        DEF_PLAYER_ID = "",
        DEF_PLAYER_NAME = "Player",
        PARTIAL_POSS = "Partial Poss",
        Odiff = "Matchup Difficulty",
    )
    .fmt_image(
        columns="DEF_PLAYER_ID",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="DEF_PLAYER_NAME")
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        table_margin_left = 1,
        table_margin_right = 1,
    )
)
t.save(fig_DIR + "Matchup_Diff_PS_2024.png",scale=3,expand=2,web_driver="edge")
t

GT(_tbl_data=   index DEF_PLAYER_ID     DEF_PLAYER_NAME  PARTIAL_POSS  Odiff
0      1        203076       Anthony Davis         409.4   4.80
1      2       1628392  Isaiah Hartenstein         287.6   4.49
2      3       1629011   Mitchell Robinson         181.0   4.41
3      4       1630529       Herbert Jones         293.0   3.33
4      5       1629060       Rui Hachimura         294.3   3.16
5      6       1629611        Terance Mann         361.3   3.10
6      7        203932        Aaron Gordon         422.3   3.02
7      8       1628389         Bam Adebayo         360.1   2.89
8      9        201976    Patrick Beverley         373.9   2.73
9     10       1630559       Austin Reaves         340.7   2.65, _body=<great_tables._gt_data.Body object at 0x0000028E65825CD0>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.default: 1>, column_label='#', column_align='center', column_width=None), ColInfo(var='DEF_PLAYER_ID', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='center', column_width=None), ColInfo(var='DEF_PLAYER_NAME', type=<ColInfoTypeEnum.default: 1>, column_label='Player', column_align='left', column_width=None), ColInfo(var='PARTIAL_POSS', type=<ColInfoTypeEnum.default: 1>, column_label='Partial Poss', column_align='center', column_width=None), ColInfo(var='Odiff', type=<ColInfoTypeEnum.default: 1>, column_label='Matchup Difficulty', column_align='center', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=6, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=7, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=8, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=9, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title=Text(text='Highest Matchup Difficulty: Weighted by Regular Season OBPM', type='from_markdown'), subtitle='Min 50 Partial Possessions Defended, ', preheader=None), _stubhead=None, _source_notes=['@SravanNBA | source: nba.com/stats via nba_api'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000028E637EAE10>, _formats=[<great_tables._gt_data.FormatInfo object at 0x0000028E637E9CD0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value=1), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value=1), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='floralwhite'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['Consolas']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value=10), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table',